In [3]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()


True

In [4]:
# utility to return (artist_id , song/album id )
def extract_ids(rows,arr):
    artists,id = rows
    for artist in artists:
        arr.append((artist['id'],id))

In [ ]:
ENDPOINT = "https://api.spotify.com/v1/me/tracks"
HEADERS={f"Authorization":f"Bearer  {os.getenv("ACCESS_TOKEN")}"}
ALBUMS_PATH = "csv/albums.csv"
ARTISTS_PATH = "csv/artists.csv"
SONGS_PATH = "csv/songs.csv"

collection , art_song_pairs , alb_art_pairs = [] , [] , []

In [ ]:
# API request to get all songs in 'liked songs'. Be care to ensure limit + offset get every song in the playlist
# Collection is a list of songs with all info attached
# i.e doesn't stop short
for offset in range(0,5):
    params = {'limit' : 49, 'offset' : offset*50}
    response = requests.get(ENDPOINT,headers=HEADERS,params=params)
    print(response.status_code)
    for x in response.json()['items']:
        collection.append(x['track'])

In [ ]:
## Get Albums database
dfs_albums = [x['album'] for x in collection]
albums = pd.DataFrame(df for df in dfs_albums)
albums[['artists','id']].apply(lambda x : extract_ids(x,alb_art_pairs),axis=1) # for albums-artists
albums['external_urls'] = albums['external_urls'].apply(lambda x : x['spotify'])
albums.drop(columns=['artists'],inplace=True,axis=1)
albums.to_csv(path_or_buf = ALBUMS_PATH)

In [ ]:
## Gets artist - album database
art_albs = pd.DataFrame(alb_art_pairs,columns=["ArtistId","AlbumId"])
art_albs.to_csv(path_or_buf = "artist_albums.csv")

In [ ]:
# Gets artists database
# Note has to work slightly different as deals with a lists of artists for each song
artists =  []
dfs_artists = [x['artists'] for x in collection]
for x in dfs_artists:
    for artist in x:
        artists.append(pd.DataFrame(artist))
artists_expanded = pd.concat(artists)
artists_expanded.to_csv(path_or_buf = ARTISTS_PATH)

In [ ]:
# Gets songs database
songs = pd.DataFrame(df for df in collection)
songs['album_id']= songs['album'].apply(lambda x : x['id'])
songs[['artists','id']].apply(lambda x : extract_ids(x,art_song_pairs),axis=1)
songs['external_urls'] = songs['external_urls'].apply(lambda x : x['spotify'])
songs['isrc'] = songs['external_ids'].apply(lambda x : x['isrc'])
songs.drop(columns=['album','artists','external_ids'],inplace=True,axis=1)
songs.to_csv(path_or_buf = SONGS_PATH)

In [ ]:
# Gets artist - song database
artists_songs = pd.DataFrame(art_song_pairs,columns=["ArtistId","SongId"])
artists_songs.to_csv(path_or_buf = "artist_songs_pairs.csv")